In [1]:
from contextlib import asynccontextmanager
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import JSONResponse
from PIL import Image
import torch
import torchvision.transforms as transforms
from google.cloud import storage
import io
from model import ResNet18
import os

In [2]:
def upload_to_gcp_bucket(bucket_name, source_file_name, destination_blob_name, key_file):
    """
    上传文件到 GCP 的 Cloud Storage。
    :param bucket_name: GCP 存储桶的名称。
    :param source_file_name: 本地文件路径。
    :param destination_blob_name: 存储到存储桶中的目标文件路径。
    :param key_file: 服务账号的密钥文件路径。
    """
    # 设置 Google Cloud 的认证环境变量
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_file

    # 初始化存储客户端
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # 上传文件
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

In [4]:
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), "../../"))

bucket_name = "mlops-trained-models"  # 替换为你的存储桶名称
source_file_name =  os.path.join(project_root, "models/model.pth")  # 替换为模型文件的本地路径
destination_blob_name = "models/model.pth"  # 替换为存储路径
key_file = os.path.join(project_root, "keys/mlops-groupwork-7a024d65ea13.json")   # 服务账号密钥文件的路径
upload_to_gcp_bucket(bucket_name, source_file_name, destination_blob_name, key_file)

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
'''
Read the image
'''
image_bytes = await file.read()
image = Image.open(io.BytesIO(image_bytes)).convert('RGB')


'''
process the image and make predictions
'''
input_tensor = preprocess_image(image) 
with torch.no_grad():
    outputs = model(input_tensor)
    probs = torch.sigmoid(outputs).squeeze().item()
category = "hotdog" if probs > 0.5 else "not hotdog"

return JSONResponse({
    'probabilities': float(probs),
    'category': category
})